In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import io
import re
import datetime

In [ ]:
def func_extract_info(function_call,symbol,datatype):
    df_1=pd.DataFrame([])
    for i in range(len(symbol)):
        url= 'https://www.alphavantage.co/query?function={}&symbol={}&datatype={}&apikey=D2L2TRRTVX26H02F'.format(function_call,symbol[i],datatype)
        r=requests.get(url=url)
        data=io.StringIO(r.text)
        df=pd.read_csv(data)
        df['Symbol']=symbol[i]
        df['timestamp']=pd.to_datetime(df['timestamp'])
        df_1=pd.concat([df_1,df])
    return df_1

In [ ]:
df_stock=func_extract_info(function_call='TIME_SERIES_MONTHLY_ADJUSTED',symbol=['BMY','PFE','GOOG','AAPL','AZN','GILD','AMGN','NVS'],datatype='csv')

In [ ]:
df_stock.to_csv('stock_data/df_stock.csv')

# Data Manipulation

In [2]:
df_stock=pd.read_csv('stock_data/df_stock.csv',index_col=0)

In [3]:
df_stock.set_index(keys=['timestamp','Symbol'],inplace=True)

In [4]:
df_stock.sort_values(by=['Symbol','timestamp'],inplace=True)

In [5]:
df_stock

,,open,high,low,close,adjusted close,volume,dividend amount
timestamp,Symbol,,,,,,,
1999-12-31,AAPL,101.0000,118.00,91.060,102.81,0.7751,84091200,0.00
2000-01-31,AAPL,104.8700,121.50,86.500,103.75,0.7822,112099800,0.00
2000-02-29,AAPL,104.0000,119.94,97.000,114.62,0.8642,65355200,0.00
2000-03-31,AAPL,118.5600,150.38,114.000,135.81,1.0239,77663900,0.00
2000-04-28,AAPL,135.5000,139.50,104.870,124.06,0.9353,77342900,0.00
...,...,...,...,...,...,...,...,...
2024-04-30,PFE,27.7600,27.87,25.200,25.62,24.9038,800213008,0.00
2024-05-31,PFE,26.3295,29.63,26.020,28.66,28.2741,954100101,0.42
2024-06-28,PFE,28.6600,29.73,26.870,27.98,27.6032,635528718,0.00


In [6]:
df_stock_roll_2=df_stock['close'].rolling(2).mean().round(2).to_frame()

In [7]:
df_stock_roll_2

,,close
timestamp,Symbol,
1999-12-31,AAPL,NaN
2000-01-31,AAPL,103.28
2000-02-29,AAPL,109.18
2000-03-31,AAPL,125.22
2000-04-28,AAPL,129.94
...,...,...
2024-04-30,PFE,26.69
2024-05-31,PFE,27.14
2024-06-28,PFE,28.32


In [11]:
df_stock_roll_2.rename(columns={'close':'2MR'},inplace=True,errors='raise')

In [12]:
df_stock_roll_2

,,2MR
timestamp,Symbol,
1999-12-31,AAPL,NaN
2000-01-31,AAPL,103.28
2000-02-29,AAPL,109.18
2000-03-31,AAPL,125.22
2000-04-28,AAPL,129.94
...,...,...
2024-04-30,PFE,26.69
2024-05-31,PFE,27.14
2024-06-28,PFE,28.32


In [13]:
df_viz=pd.merge(left=df_stock_roll_2,right=df_stock,left_index=True,right_index=True)

In [14]:
df_viz

,,2MR,open,high,low,close,adjusted close,volume,dividend amount
timestamp,Symbol,,,,,,,,
1999-12-31,AAPL,NaN,101.0000,118.00,91.060,102.81,0.7751,84091200,0.00
2000-01-31,AAPL,103.28,104.8700,121.50,86.500,103.75,0.7822,112099800,0.00
2000-02-29,AAPL,109.18,104.0000,119.94,97.000,114.62,0.8642,65355200,0.00
2000-03-31,AAPL,125.22,118.5600,150.38,114.000,135.81,1.0239,77663900,0.00
2000-04-28,AAPL,129.94,135.5000,139.50,104.870,124.06,0.9353,77342900,0.00
...,...,...,...,...,...,...,...,...,...
2024-04-30,PFE,26.69,27.7600,27.87,25.200,25.62,24.9038,800213008,0.00
2024-05-31,PFE,27.14,26.3295,29.63,26.020,28.66,28.2741,954100101,0.42
2024-06-28,PFE,28.32,28.6600,29.73,26.870,27.98,27.6032,635528718,0.00


# Visualization

In [ ]:
from bokeh.plotting import figure,show
from bokeh.layouts import gridplot

In [ ]:
variable_name=[]
for i in set(df_stock['Symbol']):
   Dynamic_Variable_Name='p_'+i
   locals()[Dynamic_Variable_Name]=figure(title=i,x_axis_label='TimeStamp',y_axis_label='Price',x_axis_type='datetime',height=480,width=800)
   locals()[Dynamic_Variable_Name].line(x=df_stock[df_stock['Symbol']==i]['timestamp'],y=df_stock[df_stock['Symbol']==i]['close'])
   #locals()[Dynamic_Variable_Name].line(x=df_stock_roll_2[df_stock_roll_2['Symbol']==j]['timestamp'],y=df_stock_roll_2[df_stock_roll_2['Symbol']==j]['open'])
   variable_name.append(Dynamic_Variable_Name)

In [ ]:
test=vars()

In [ ]:
list_fig=[test[var] for var in variable_name]

In [ ]:
grid=gridplot(list_fig,ncols=4,sizing_mode='scale_both')

In [ ]:
show(grid)